In [1]:
import sys
sys.path.append('../main')
import sql
import pandas as pd
from functools import reduce

In [9]:
q='''select name, count(*) as total from alert a
        join project p on a.project_id = p.id
        join memory_error me on a.alert_type_id = me.alert_type_id
        where is_invalid=0
        group by name
        order by name'''
ta = pd.DataFrame(sql.execute(q)) 
q='''select name, count(*) as memory from alert a
        join project p on a.project_id = p.id
        join memory_error me on a.alert_type_id = me.alert_type_id
        where is_invalid=0
        and me.memory =1 
        group by name
        order by name'''
tma = pd.DataFrame(sql.execute(q)) 
tma['memory (%)']= round((tma.memory / ta.total) * 100,2)
q='''select name, count(*) as non_memory from alert a
        join project p on a.project_id = p.id
        join memory_error me on a.alert_type_id = me.alert_type_id
        where is_invalid=0
        and me.memory is null
        group by name
        order by name'''
tnma = pd.DataFrame(sql.execute(q)) 
tnma['non_memory (%)']= round((tnma.non_memory / ta.total) * 100,2)
df= reduce(lambda x,y: pd.merge(x,y, on='name', how='outer'),[ta,tma,tnma])
df['memory (%)']=df['memory (%)'].astype(str)+"%"
df['non_memory (%)']=df['non_memory (%)'].astype(str)+"%"
df = df.sort_values('total',ascending=False)
df

,name,total,memory,memory (%),non_memory,non_memory (%)
5,Linux,19514,7903,40.5%,11611,59.5%
2,Firefox,12945,5448,42.09%,7497,57.91%
4,LibreOffice,12495,4174,33.41%,8321,66.59%
8,Samba,4507,1700,37.72%,2807,62.28%
7,ovirt-engine,2989,1272,42.56%,1717,57.44%
10,VTK,2522,1166,46.23%,1356,53.77%
6,OpenCV,2443,710,29.06%,1733,70.94%
3,Kodi,2393,682,28.5%,1711,71.5%
0,!CHAOS Control System,1980,282,14.24%,1698,85.76%
1,Chromium EC,964,306,31.74%,658,68.26%


In [3]:
q='''select * from alert a
join project p on a.project_id = p.id
join memory_error me on a.alert_type_id = me.alert_type_id
where is_invalid=0
and me.memory =1
order by name;'''
root=pd.DataFrame(sql.execute(q)) 
#root.head()
total=root.groupby('name')[['cid']].count()
total.rename(columns={'cid':'total'},inplace=True)
eliminated=root[root['status']=='Fixed'].groupby('name')[['cid']].count()
eliminated.rename(columns={'cid':'e'},inplace=True)
bug=root[root['classification']=='Bug'].groupby('name')[['cid']].count()
q='''select * from alert a
join project p on a.project_id = p.id
join memory_error me on a.alert_type_id = me.alert_type_id
join actionability ac on ac.alert_id=a.id
where is_invalid=0
and actionability = 1
and me.memory =1
order by name;'''
df=pd.DataFrame(sql.execute(q))
actionable=df.groupby('name')[['cid']].count()
actionable.rename(columns={'cid':'a'},inplace=True)
bug.rename(columns={'cid':'b'},inplace=True)
df=reduce(lambda x,y:pd.merge(x,y,on='name'),[total,eliminated, actionable, bug])
df.e = round((df.e/df.total)*100,2)    
df.a = round((df.a/df.total)*100,2)
df.b = round((df.b/df.total)*100,2)   
df.e=df.e.astype(str)+'%'
df.a=df.a.astype(str)+'%'
df.b=df.b.astype(str)+'%'
df.rename(columns={'total':'total memory', 'e':'eliminated memory', 'a':'actionable memory','b':'triaged_bug memory'}, inplace=True)
memory = df        
memory = memory.sort_values('total memory', ascending=False)
memory                                               

,total memory,eliminated memory,actionable memory,triaged_bug memory
name,,,,
Linux,7903,73.06%,43.54%,3.11%
Firefox,5448,77.22%,49.72%,12.43%
LibreOffice,4174,80.57%,73.74%,52.68%
Samba,1700,77.88%,39.88%,3.82%
ovirt-engine,1272,36.08%,27.12%,2.59%
VTK,1166,12.78%,7.2%,0.43%
OpenCV,710,89.15%,32.54%,0.28%
Kodi,682,68.62%,54.11%,25.07%
Thunderbird,357,63.31%,24.37%,12.61%


In [4]:
q='''select * from alert a
join project p on a.project_id = p.id
join memory_error me on a.alert_type_id = me.alert_type_id
where is_invalid=0
and me.memory is null
order by name;'''
root=pd.DataFrame(sql.execute(q)) 
#root.head()
total=root.groupby('name')[['cid']].count()
total.rename(columns={'cid':'total'},inplace=True)
eliminated=root[root['status']=='Fixed'].groupby('name')[['cid']].count()
eliminated.rename(columns={'cid':'e'},inplace=True)
bug=root[root['classification']=='Bug'].groupby('name')[['cid']].count()
q='''select * from alert a
join project p on a.project_id = p.id
join memory_error me on a.alert_type_id = me.alert_type_id
join actionability ac on ac.alert_id=a.id
where is_invalid=0
and actionability = 1
and me.memory is null
order by name;'''
df=pd.DataFrame(sql.execute(q))
actionable=df.groupby('name')[['cid']].count()
actionable.rename(columns={'cid':'a'},inplace=True)
bug.rename(columns={'cid':'b'},inplace=True)
df=reduce(lambda x,y:pd.merge(x,y,on='name'),[total,eliminated, actionable, bug])
df.e = round((df.e/df.total)*100,2)    
df.a = round((df.a/df.total)*100,2)
df.b = round((df.b/df.total)*100,2)   
df.e=df.e.astype(str)+'%'
df.a=df.a.astype(str)+'%'
df.b=df.b.astype(str)+'%'
df.rename(columns={'total':'total non_memory', 'e':'eliminated non_memory', 'a':'actionable non_memory','b':'triaged_bug non_memory'}, inplace=True)
non_memory = df        
non_memory = non_memory.sort_values('total non_memory', ascending=False) 

In [5]:
df=reduce(lambda x,y: pd.merge(x,y,on='name'),[memory,non_memory])
df=df[['total memory', 'total non_memory', 'eliminated memory', 'eliminated non_memory',
    'actionable memory','actionable non_memory','triaged_bug memory','triaged_bug non_memory']]
df = df.sort_values('total memory', ascending=False)
df

,total memory,total non_memory,eliminated memory,eliminated non_memory,actionable memory,actionable non_memory,triaged_bug memory,triaged_bug non_memory
name,,,,,,,,
Linux,7903,11611,73.06%,65.19%,43.54%,39.08%,3.11%,3.42%
Firefox,5448,7497,77.22%,70.9%,49.72%,46.47%,12.43%,5.14%
LibreOffice,4174,8321,80.57%,94.08%,73.74%,71.01%,52.68%,41.39%
Samba,1700,2807,77.88%,73.35%,39.88%,24.72%,3.82%,1.53%
ovirt-engine,1272,1717,36.08%,47.82%,27.12%,32.62%,2.59%,2.56%
VTK,1166,1356,12.78%,19.1%,7.2%,7.6%,0.43%,1.84%
OpenCV,710,1733,89.15%,76.46%,32.54%,35.95%,0.28%,0.4%
Kodi,682,1711,68.62%,67.27%,54.11%,50.32%,25.07%,11.57%
Thunderbird,357,528,63.31%,49.81%,24.37%,14.96%,12.61%,5.11%
